<a href="https://colab.research.google.com/github/xechehot/sirius-2021-warehouse/blob/main/Maksim/beta_version1.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def gen_data(N, M):
    data = [[rand(0, N - 1), rand(0, M - 1)] for _ in range(10)]
    block = numpy.zeros((N, M), int)
    for pas, sec in data:
        block[pas, sec] += 1
    return block

In [115]:
def get_dist_for_1_path(sx, sy, block, data_for_ans): #data_for_ans - для ячейки храним какой заказ в ней
    x = sx
    y = sy
    dist = 0
    ans = []
    d = -1
    N = len(block)
    M = len(block[0])
    if (sy == 0): d = 1
    while True:
        if x < 0 or y < 0 or x >= N or y >= M: break
        if (block[:, y].sum() == 0):
            dist += 2
            y += d
            continue
        if x == N - 1: #идем вверх
            for nx in range(N - 1, -1, -1):
                if block[nx][y] != 0: 
                    ans.append(data_for_ans[nx][y])
        else: # идем вниз
            for nx in range(0, N):
                if (block[nx][y] != 0):
                    ans.append(data_for_ans[nx][y])
        dist += N
        dist += 2
        x = abs(x - (N - 1))
        y += d
    dist -= 2

    if x == N - 1: dist += N
    return [dist, ans]




In [3]:
from random import randint as rand
import numpy
import json

In [6]:
! git clone https://github.com/xechehot/sirius-2021-warehouse.git

fatal: destination path 'sirius-2021-warehouse' already exists and is not an empty directory.


In [45]:
with open('sirius-2021-warehouse/data_sample/sample.json') as f:
    sample_data = json.load(f)

In [96]:
rows = sample_data['warehouse']['meta']['rows']
sections = sample_data['warehouse']['meta']['sections']
block_x = sample_data['warehouse']['meta']['block_x']
block_y = sample_data['warehouse']['meta']['block_y']
floors = sample_data['warehouse']['meta']['floors']
batch = sample_data['batch_size']
b_sz = 32
N = rows // 2 + 1
M = sections

In [75]:
def get_places(cnt_products, product_place): #ищу ячейки, куда нужно зайти
    ans = []
    for i in range(len(product_place)):
        product_place[i].sort(key = lambda a: -a['count'])
    for i in range(pcnt):
        have = cnt_products[i]
        ind = 0
        while (have > 0 and ind < len(product_place[i])):
            if (product_place[i][ind]['count'] >= have):
                ans.append([product_place[i][ind]['p'], have])
                have = 0
            else:
                ans.append([product_place[i][ind]['p'], product_place[i][ind]['count']])
                have -= product_place[i][ind]['count']
            ind += 1
    return ans

In [83]:
def distribute_data(places): #разделяю ячейки по этажам, блокам 
    ans = [[[[] for i1 in range(block_y)] for i2 in range(block_x)] for i3 in range(floors)]
    for place in places:
        ans[place[0]['floor'] - 1][place[0]['block_x'] - 1][place[0]['block_y'] - 1].append(place)
    return ans

In [99]:
def conv_place(place):
    return [place['row'] // 2, place['section']]

In [131]:
def dist_to_start(place):
    ans = (sections * block_x + 2) * 2
    ans -= sections
    dy = int(abs(1.5 - place['block_y']) + 0.5)
    ans += dy * rows
    if (place['block_y'] <= 1): ans += (rows - place['row'])
    ans += (sections + 1 - place['section'])
    return ans

In [123]:
def get_ans_for_1_block(data, sx, sy):
    block = numpy.zeros((N, M), int)
    list_of_products = [[-1 for i in range(M)] for j in range(N)]
    for place, cnt in data:
        x, y = conv_place(place)
        block[x, y] += cnt
        list_of_products[x][y] = [place, cnt]
    dist, paths = get_dist_for_1_path(sx, sy, block, list_of_products)
    ans = []
    cur = []
    sum = 0
    for place, cnt in paths:
        if (sum + cnt <= 32):
            cur.append([place, cnt])
            sum += cnt
        else:
            exist = cnt
            while exist > 0:
                now = min(exist, 32 - sum)
                cur.append([place, now])
                sum += now
                exist -= now
                if (sum == 32):
                    dist += place['section'] * 2
                    dist += dist_to_start(place)*2
                    ans.append(cur)
                    cur = []
                    sum = 0
    if (cur != []):
        dist += dist_to_start(place)
        ans.append(cur)
        cur = []
        sum = 0
    return [ans, dist]
    



In [93]:
pcnt = 0 # количество различных товаров
for order in sample_data['orders']:
    for product in order['items']:
        pcnt = max(pcnt, product['id'] + 1)
product_place = [[] for i in range(pcnt)]
for product in sample_data['warehouse']['stock']:
    product_place[product['id']].append(product)
cnt_products = [0 for i in range(pcnt)]
for order in sample_data['orders']:
    for product in order['items']:
        cnt_products[product['id']] += product['count']


In [132]:
places = get_places(cnt_products, product_place)
dis = distribute_data(places)
ans = []
dist = 0
for floor in range(floors):
    for bx in range(block_x):
        for by in range(block_y):
            if (dis[floor][bx][by] == []): continue
            sx = N - 1
            if (by >= 2): sx = 0
            p_ans, p_dist = get_ans_for_1_block(dis[floor][bx][by], sx, 0)
            dist += p_dist
            for elem in p_ans: ans.append(elem)
print(ans)
print(dist)

[[[{'floor': 1, 'block_x': 1, 'block_y': 2, 'row': 8, 'section': 2}, 3]], [[{'floor': 1, 'block_x': 1, 'block_y': 3, 'row': 1, 'section': 2}, 16]], [[{'floor': 1, 'block_x': 2, 'block_y': 3, 'row': 4, 'section': 9}, 4]], [[{'floor': 1, 'block_x': 2, 'block_y': 4, 'row': 7, 'section': 9}, 7]], [[{'floor': 2, 'block_x': 1, 'block_y': 2, 'row': 5, 'section': 4}, 2]], [[{'floor': 2, 'block_x': 1, 'block_y': 3, 'row': 5, 'section': 4}, 3]]]
486
